In [34]:
import requests
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [35]:
raw_data = pd.read_csv('Profeco_Catalog.csv')
raw_data.head(1)

,Unnamed: 0,_id,cadenaComercial,catalogo,categoria,direccion,estado,fechaRegistro,giro,latitud,longitud,marca,municipio,nombreComercial,precio,presentacion,producto,razonSocial,rfc
0,0,571e4c002f232201006c6cac,HIPERMERCADO SORIANA,BASICOS,TORTILLAS Y DERIVADOS DEL MAIZ,"BLVD. ADOLFO LOPEZ MATEOS 1100, ESQ. CALAFIA, ...",BAJA CALIFORNIA,2015-02-10,TIENDA DE AUTOSERVICIO,32.641719,-115.469582,S/M,MEXICALI,SORIANA HIPER SUCURSAL CALAFIA,11.9,1 KG. GRANEL,TORTILLA DE MAIZ,NaN,NaN


In [36]:
master_frame = raw_data.drop(columns = ['giro','razonSocial','rfc','catalogo','Unnamed: 0','direccion','nombreComercial']).reset_index(drop=True).rename(index=str, columns={"_id": "ID", "cadenaComercial": "Cadena", "categoria": "Categoría", "estado": "Estado", "fechaRegistro": "Fecha", "latitud": "Latitud", "longitud": "Longitud", "marca": "Marca", "municipio": "Municipio", "precio": "Precio", "presentacion": "Presentación", "producto": "Producto"})
master_frame.head(1)

,ID,Cadena,Categoría,Estado,Fecha,Latitud,Longitud,Marca,Municipio,Precio,Presentación,Producto
0,571e4c002f232201006c6cac,HIPERMERCADO SORIANA,TORTILLAS Y DERIVADOS DEL MAIZ,BAJA CALIFORNIA,2015-02-10,32.641719,-115.469582,S/M,MEXICALI,11.9,1 KG. GRANEL,TORTILLA DE MAIZ


In [37]:
year_registry = master_frame.groupby('Fecha')['Fecha'].count()
year_registry

Fecha
2012-01-10      9063
2013-01-10     40539
2015-02-10    131192
2016-05-10     38788
Name: Fecha, dtype: int64

In [61]:
master_2015frame = master_frame[(master_frame['Fecha'] == '2016-05-10')]
#master_2015frame
category = master_2015frame.groupby('Categoría')['Categoría'].count().nlargest(10)
category

Categoría
MEDICAMENTOS                         10711
APARATOS ELECTRICOS                   2968
ARTS. PARA EL CUIDADO PERSONAL        2252
APARATOS ELECTRONICOS                 2017
DERIVADOS DE LECHE                    1673
CARNES FRIAS SECAS Y EMBUTIDOS        1563
DETERGENTES Y PRODUCTOS SIMILARES     1443
REFRESCOS ENVASADOS                   1410
FRUTAS Y LEGUMBRES PROCESADAS         1207
HORTALIZAS FRESCAS                    1169
Name: Categoría, dtype: int64

In [62]:
#Estados = Master["Estado"].nunique()
Estados = master_2015frame.drop_duplicates("Estado")
#Estados.head(100)
Estados = Estados["Estado"].tolist()
Estados

['TLAXCALA',
 'YUCATÁN',
 'MÉXICO',
 'QUINTANA ROO',
 'NUEVO LEÓN',
 'SONORA',
 'JALISCO',
 'PUEBLA',
 'VERACRUZ DE IGNACIO DE LA LLAVE',
 'COLIMA',
 'GUANAJUATO',
 'TABASCO',
 'DISTRITO FEDERAL',
 'AGUASCALIENTES',
 'BAJA CALIFORNIA',
 'BAJA CALIFORNIA SUR',
 'CAMPECHE',
 'DURANGO',
 'CHIAPAS',
 'CHIHUAHUA',
 'GUERRERO',
 'TAMAULIPAS',
 'MICHOACÁN DE OCAMPO',
 'COAHUILA DE ZARAGOZA',
 'MORELOS']

In [102]:
category_state = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Estado', 'Categoría'])['Categoría'].count().nlargest(5)
#category_state
category_total = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].count()
category_total = category_total['ID']
category_total

10711

In [103]:
#master_frame.groupby(master_frame['Fecha'].date.dt.year)

## Gráfica 3
### Count de categoría por Estado

Aquí tienes que hacer un chart con el número de categorías por Top 5 de estados - Eje X = Estado Eje Y = Categoría

In [110]:
top5_category = pd.DataFrame([category_state])
top5_category.columns = top5_category.columns.droplevel(-1)
#Data Frame con el Top 5 de estados con mayores registros de medicamentos
top5_category.transpose().reset_index()


,Estado,Categoría
0,MÉXICO,1551
1,DISTRITO FEDERAL,901
2,GUANAJUATO,865
3,JALISCO,816
4,QUINTANA ROO,724


In [115]:
#Top5Chart = top5_category.plot.bar(x='Estado', y='Categoría', rot=0)

## Gráfica 4
### % del total de productos de la categoría medicamentos por Estado

Esta es una métrica que saqué aparte y creo que si es posible podríamos fusionar la primera con la gráfica3 y podemos hacer una gráfica combinada de barras y línea para que salgan ambas métricas. pero l oque sea más fácil

In [112]:
#Data Frame con el Top 5 de estados con el % de contribución del total de registros de medicamentos
top5_categoryperc = top5_category / category_total
top5_categoryperc.transpose().style.format("{:.2%}")

,Categoría
Estado,
MÉXICO,14.48%
DISTRITO FEDERAL,8.41%
GUANAJUATO,8.08%
JALISCO,7.62%
QUINTANA ROO,6.76%


## Gráfica 5
### Precio promedio de medicamentos 2015 por top 5 (max precio)

Gráfica que ponga el top 5 de los estados con mayor precio en Mexico

In [139]:
avg_medprice = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Categoría'])['Precio'].mean().nlargest(32)
#avg_medprice = avg_medprice['ID']
avg_medprice = avg_medprice['MEDICAMENTOS']
avg_medprice

355.45931752404033

In [149]:
avg_price = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Estado', 'Categoría'])['Precio'].mean().nlargest(5)
avg_price

Estado               Categoría   
DISTRITO FEDERAL     MEDICAMENTOS    428.526770
SONORA               MEDICAMENTOS    398.167700
BAJA CALIFORNIA      MEDICAMENTOS    378.842113
BAJA CALIFORNIA SUR  MEDICAMENTOS    378.107500
TABASCO              MEDICAMENTOS    372.412813
Name: Precio, dtype: float64

In [151]:
top5_price = pd.DataFrame([avg_price])
top5_price.columns = top5_price.columns.droplevel(-1)
#Data Frame con el Top 5 de estados con mayores registros de medicamentos
top5_price.transpose()

,Precio
Estado,
DISTRITO FEDERAL,428.526770
SONORA,398.167700
BAJA CALIFORNIA,378.842113
BAJA CALIFORNIA SUR,378.107500
TABASCO,372.412813


## Gráfica 6
### % prima o descuento de precio promedio en medicamentos por el top 5 de Estados con mayor precio

Esta es una métrica que saqué aparte y creo que si es posible podríamos fusionar la primera con la gráfica5 y podemos hacer una gráfica combinada de barras y línea para que salgan ambas métricas. pero l oque sea más fácil

In [153]:
top5_priceperc = top5_price / avg_medprice - 1
top5_priceperc.transpose().style.format("{:.2%}")

,Precio
Estado,
DISTRITO FEDERAL,20.56%
SONORA,12.01%
BAJA CALIFORNIA,6.58%
BAJA CALIFORNIA SUR,6.37%
TABASCO,4.77%


In [155]:
chain_category = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Cadena', 'Categoría'])['Categoría'].count().nlargest(5)
chain_category

Cadena                Categoría   
WAL-MART              MEDICAMENTOS    1580
BODEGA AURRERA        MEDICAMENTOS    1448
FARMACIA GUADALAJARA  MEDICAMENTOS     740
BENAVIDES             MEDICAMENTOS     706
HIPERMERCADO SORIANA  MEDICAMENTOS     690
Name: Categoría, dtype: int64

## Gráfica 7
### Count de categoría por Cadena top5

Aquí tienes que hacer un chart con el número de categorías por Top 5 de cadenaS - Eje X = Cadena Eje Y = Categoría

In [157]:
top5_chain = pd.DataFrame([chain_category])
top5_chain.columns = top5_chain.columns.droplevel(-1)
#Data Frame con el Top 5 de estados con mayores registros de medicamentos
top5_chain.transpose().reset_index()

,Cadena,Categoría
0,WAL-MART,1580
1,BODEGA AURRERA,1448
2,FARMACIA GUADALAJARA,740
3,BENAVIDES,706
4,HIPERMERCADO SORIANA,690


## Gráfica 8
### % del total de productos de la categoría medicamentos por Cadena Comercial

Esta es una métrica que saqué aparte y creo que si es posible podríamos fusionar la primera con la gráfica7 y podemos hacer una gráfica combinada de barras y línea para que salgan ambas métricas. pero l oque sea más fácil

In [160]:
top5_chainperc = top5_chain / category_total
top5_chainperc.transpose().style.format("{:.2%}")

,Categoría
Cadena,
WAL-MART,14.75%
BODEGA AURRERA,13.52%
FARMACIA GUADALAJARA,6.91%
BENAVIDES,6.59%
HIPERMERCADO SORIANA,6.44%


In [168]:
chain_price = master_2015frame[master_2015frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Cadena'])['Precio'].mean().nlargest(5)
chain_price

Cadena
FARMACIA DEL HOSPITAL     540.297217
SANBORN S HNOS.           532.554239
FARMATODO                 414.097855
SORIANA                   383.970820
FARMACIA LA MAS BARATA    383.771878
Name: Precio, dtype: float64

## Gráfica 9
### Precio promedio de medicamentos 2015 por top 5 cadenas (max precio)

Gráfica que ponga el top 5 de las cadenas con mayor precio en Mexico

In [169]:
top5_chainprice = pd.DataFrame([chain_price])
#top5_chainprice.columns = top5_chain.columns.droplevel(-1)
#Data Frame con el Top 5 de estados con mayores registros de medicamentos
top5_chainprice.transpose()

,Precio
Cadena,
FARMACIA DEL HOSPITAL,540.297217
SANBORN S HNOS.,532.554239
FARMATODO,414.097855
SORIANA,383.970820
FARMACIA LA MAS BARATA,383.771878


## Gráfica 10
### % prima o descuento de precio promedio en medicamentos por el top 5 de Cadenas con mayor precio

Esta es una métrica que saqué aparte y creo que si es posible podríamos fusionar la primera con la gráfica9 y podemos hacer una gráfica combinada de barras y línea para que salgan ambas métricas. pero l oque sea más fácil

In [171]:
top5_chainpriceperc = top5_chainprice / avg_medprice - 1
top5_chainpriceperc.transpose().style.format("{:.2%}")

,Precio
Cadena,
FARMACIA DEL HOSPITAL,52.00%
SANBORN S HNOS.,49.82%
FARMATODO,16.50%
SORIANA,8.02%
FARMACIA LA MAS BARATA,7.97%


# A huevo, nos la pela esto